In [2]:
from read_api_keys import read_api_keys
import itertools
import numpy as np
import pandas as pd
import time
import sys
sys.path.append('../')
from pybit_ms.bybit_client import BybitAPI

We first

In [3]:
keys = read_api_keys("TAK.txt")
public_key = keys.get('PUBLIC_KEY')
private_key = keys.get('PRIVATE_KEY')

client = BybitAPI(api_key=public_key, api_secret=private_key, testnet=1)
client

BybitAPI(testnet=1)

In [4]:
COINS = ["BTC", "ETH", "BNB", "SOL", "XRP", "DOGE", "ADA", "LINK", "DOT", "TRX"]
COIN_PAIRS = list(itertools.combinations(COINS, 2))

In [5]:
data = pd.DataFrame()

for coin in COINS:
    # Get Kline of previous 30 days with interval of 1 hour
    data[f'{coin}'] = client.market.get_kline(category="linear", coin1=coin, coin2="USDT", interval="60", price_type="close", limit=720)
    
    # Convert prices to float
    data[f'{coin}'] = pd.to_numeric(data[f'{coin}'], errors='coerce')
    
    # Normalize the prices
    min_price = data[f'{coin}'].min()
    max_price = data[f'{coin}'].max()
    data[f'{coin}'] = (data[f'{coin}']) - min_price / (max_price - min_price)
    
data

,BTC,ETH,BNB,SOL,XRP,DOGE,ADA,LINK,DOT,TRX
0,86866.599128,3315.278416,698.726044,176.056583,0.202225,-1.149687,-0.752262,17.382702,5.0663,-1.409841
1,85499.999128,3346.468416,676.726044,191.626583,0.218225,-1.148977,-0.697762,18.680702,4.6103,-1.418511
2,30475.499128,3172.868416,676.726044,187.216583,0.187625,-1.155707,-0.823162,17.091702,5.1553,-1.425111
3,39359.999128,3288.858416,684.726044,195.486583,0.153225,-1.148317,-0.793362,18.959702,5.0613,-1.426961
4,47974.699128,3253.798416,688.226044,196.096583,0.161125,-1.109017,-0.724762,20.667702,4.9893,-1.412331
...,...,...,...,...,...,...,...,...,...,...
715,74319.999128,3639.148416,654.726044,215.946583,0.185925,-1.068317,-0.723662,18.603702,6.6783,-1.377211
716,74319.999128,3692.658416,662.226044,207.226583,0.324825,-1.065587,-0.611762,20.549702,6.4113,-1.426621
717,74330.499128,3559.718416,678.226044,212.676583,0.340225,-1.042357,-0.601462,20.172702,6.7593,-1.393181
718,74519.899128,3599.958416,656.726044,210.946583,0.291025,-1.071647,-0.682662,20.640702,6.4403,-1.386811


In [ ]:
!pip install statsmodels

In [6]:
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

def calc_adf(data, coin1, coin2):
    Y = data[f'{coin1}']
    X = data[f'{coin2}']
    X = sm.add_constant(X)  
    model = sm.OLS(Y, X).fit()
    residuals = model.resid
    result = adfuller(residuals)[0]
    return result

# Find pairs with highest cointegration
def coint_pairs(data):  
    adfs = {}
    for pair in COIN_PAIRS:
        coin1 = pair[0]
        coin2 = pair[1]
        adfs[f"{coin1}-{coin2}"] = calc_adf(data, coin1, coin2)
    adfs = dict(sorted(adfs.items(), key=lambda item: item[1], reverse=True))
    return list(adfs.items())

top_pair = coint_pairs(data)[:1][0]
print(top_pair)
coin1 = top_pair[0][:top_pair[0].index('-')]
coin2 = top_pair[0][top_pair[0].index('-')+1:]


('DOGE-LINK', np.float64(-1.1314389267062768))


In [7]:
client.account.get_wallet_balance(accountType="UNIFIED")

Total equity: $102,852.27
BTC: Wallet Balance = 1.000000, USD Value = $92928.82
USDT: Wallet Balance = 9978.951959, USD Value = $9923.45


In [ ]:
q = 1000
open_position=False

while True:
	coin1_past_5days = client.market.get_kline(category="linear", coin1=coin1, coin2="USDT", interval="60", price_type="close", limit=120)
	coin2_past_5days = client.market.get_kline(category="linear", coin1=coin2, coin2="USDT", interval="60", price_type="close", limit=120)
	
	coin1_past_5days = np.array(coin1_past_5days, dtype=float)
	coin2_past_5days = np.array(coin2_past_5days, dtype=float)
	
	min_price1 = coin1_past_5days.min()
	max_price1 = coin1_past_5days.max()
	coin1_past_5days = (coin1_past_5days - min_price1) / (max_price1 - min_price1)
	
	min_price2 = coin2_past_5days.min()
	max_price2 = coin2_past_5days.max()
	coin2_past_5days = (coin2_past_5days - min_price2) / (max_price2 - min_price2)
	
	spread = coin2_past_5days - coin1_past_5days
	spread_mean = spread.mean()
	spread_std = spread.std()
	
	up_band = spread_mean + 2 * spread_std
	low_band = spread_mean - 2 * spread_std

	print("Calculated stats...")
	

	coin1_price = client.market.get_tickers(category="linear", symbol=f"{coin1}USDT", only_ticker=True)
	coin2_price = client.market.get_tickers(category="linear", symbol=f"{coin2}USDT", only_ticker=True)

	
	if spread[-1] < low_band and not open_position:
		client.trade.place_futures_order(symbol=f"{coin1}USDT", side="Sell", order_type="Market", qty=q, market_unit="quoteCoin")
		client.trade.place_futures_order(symbol=f"{coin2}USDT", side="Buy", order_type="Market", qty=q, market_unit="quoteCoin")
		open_position = True
		
	if spread[-1] > up_band and not open_position:
		client.trade.place_futures_order(symbol=f"{coin1}USDT", side="Buy", order_type="Market", qty=q, market_unit="quoteCoin")
		client.trade.place_futures_order(symbol=f"{coin2}USDT", side="Sell", order_type="Market", qty=q, market_unit="quoteCoin")
		open_position = True

	if spread[-2] > spread_mean and spread[-1] < spread_mean and open_position:
		client.trade.close_order(symbol=f"{coin1}USDT", side="Sell", order_type="Market", qty=q, market_unit="quoteCoin")
		client.trade.close_order(symbol=f"{coin2}USDT", side="Buy", order_type="Market", qty=q, market_unit="quoteCoin")
		open_position = False
	
	if spread[-2] < spread_mean and spread[-1] > spread_mean  and open_position:
		client.trade.close_order(symbol=f"{coin1}USDT", side="Buy", order_type="Market", qty=q, market_unit="quoteCoin")
		client.trade.close_order(symbol=f"{coin2}USDT", side="Sell", order_type="Market", qty=q, market_unit="quoteCoin")
		open_position = False

	client.trade.get_positions(category="linear", settle_coin="USDT")
	print("Waiting next hour")
	time.sleep(3600)


In [8]:
client.trade.cancel_all_orders(category="linear", settle_coin="USDT")

['b7676bba-9e95-46b2-9e1b-9289b02b6a89']

In [9]:
client.trade.get_closed_pnl(category="linear")

,symbol,orderType,side,leverage,orderPrice,avgEntryPrice,avgExitPrice,qty,closedSize,closedPnl,fillCount,orderId,createdTime
0,BTCUSDT,Trade (Market),Buy,10,84538.1,83075.38,82528.1,0.05,0.05,7.17628741,1,id: 69b9ef3a-a644-4cba-b02e-c29cdeb665b0,2025-01-09 00:02:55
